In [73]:
import os
import pandas as pd
csv_files_path = "../../data/forms"

forms_generated_path = "../../generated/forms"
if not os.path.exists(forms_generated_path):
    os.makedirs(forms_generated_path)

output_csv_file = f"{forms_generated_path}/data.csv"
invalid_data_csv_file = f"{forms_generated_path}/invalid-data.csv"

csv_files = ["demographic.csv", "achievements.csv", "leaderboard.csv", 'satisfaction.csv']

outer_merged_df = None
inner_merged_df = None
id_key = 'Identifiant'


def group_set(row):
    identifier = row[id_key]

    if identifier.isnumeric():
        return "A" if int(identifier) % 2 != 0 else "B"

    return "Invalid"


for i, csv_file in enumerate(csv_files):
    file_path = os.path.join(csv_files_path, csv_file)
    df = pd.read_csv(file_path, header=0)

    # Rename columns to add file initial
    initial = csv_file[0].upper()
    df.columns = [col if "Horodateur" in col else (f"{initial}-{col}" if col != id_key else col) for col in df.columns]

    duplicated = df[df.duplicated(subset=[id_key], keep=False)]
    duplicated_count = len(duplicated) / 2

    if duplicated_count > 0:
        print(f"⚠️ {csv_file}: {duplicated_count} duplicate(s), keep only firsts")
        print(duplicated[id_key])
        print("")

    df = df.drop_duplicates(subset=[id_key], keep="last")

    if id_key not in df.columns:
        print(f"⚠️ '{id_key}' not found in {csv_file}.")
        print(f"Available columns : {df.columns.tolist()}")
        continue

    if outer_merged_df is None:
        outer_merged_df = df
    else:
        outer_merged_df = pd.merge(outer_merged_df, df, on=id_key, how='outer')

    if inner_merged_df is None:
        inner_merged_df = df
    else:
        inner_merged_df = pd.merge(inner_merged_df, df, on=id_key, how='inner')

inner_merged_df.insert(1, "group", inner_merged_df.apply(group_set, axis=1))
outer_merged_df.insert(1, "group", outer_merged_df.apply(group_set, axis=1))


⚠️ demographic.csv: 2.0 duplicate(s), keep only firsts
33    86
39    56
42    86
43    56
Name: Identifiant, dtype: object

⚠️ achievements.csv: 1.0 duplicate(s), keep only firsts
20    86
41    86
Name: Identifiant, dtype: object



## Differences

In [74]:
inner_outer_merged_df = pd.merge(inner_merged_df, outer_merged_df, how='outer', indicator=True)

deleted_rows = inner_outer_merged_df[inner_outer_merged_df['_merge'] != 'both']

columns_to_keep = ["Identifiant", "group"] + list(deleted_rows.loc[:, deleted_rows.columns.str.contains("Horodateur")])

not_filled = deleted_rows[columns_to_keep].fillna("Not filled")
not_filled.to_csv(invalid_data_csv_file, index=False)
not_filled

,Identifiant,group,Demographic Horodateur,Achievements Horodateur,Leaderboard Horodateur,Satisfaction Horodateur
6,2,B,17/04/2025 08:50:26,17/04/2025 10:07:05,17/04/2025 09:31:29,Not filled
16,39,A,17/04/2025 08:48:02,Not filled,Not filled,Not filled
25,67,A,17/04/2025 08:49:09,17/04/2025 09:33:46,Not filled,Not filled
27,69,A,Not filled,17/04/2025 09:33:44,17/04/2025 10:09:46,Not filled
32,77,A,17/04/2025 08:48:24,17/04/2025 09:34:04,Not filled,Not filled
35,82,B,Not filled,17/04/2025 10:09:09,17/04/2025 09:32:49,Not filled
37,86,B,17/04/2025 09:31:47,17/04/2025 10:09:09,Not filled,Not filled
42,Selice,Invalid,17/04/2025 08:48:26,Not filled,Not filled,Not filled
43,hicarlie,Invalid,17/04/2025 08:48:34,Not filled,17/04/2025 09:30:56,Not filled


## Merged data

In [75]:
inner_merged_df

,Identifiant,group,Demographic Horodateur,D-Année d'étude,D-Genre,D-Outils gamifiés déjà utilisés ?,D-Compétences java,D-Compétences en écriture de test,D-Motivation initiale à écrire des tests,Achievements Horodateur,...,"S-Clarté des règles du jeu (achievements, leaderboard)",S-Design et intégration dans IntelliJ,"S-Selon vous, quel mode est le plus fun ?",S-Le système de points et de progression vous a-t-il semblé juste ?,"S-J'ai été tenté d'écrire plus de tests pour augmenter mes points, même si les tests ne sont pas spécialement pertinents","S-Seriez-vous intéressé·e par l'utilisation d'outils gamifiés durant certains cours ?\n(Vérification et Validation, Génie Logiciel, ..)","S-Quel mode j'ai envie d'utiliser dans un cours où je peux être confronté à du test logiciel ? \n(Vérification et Validation, Génie Logiciel, ..)",S-Donner 3 points positifs du plugin,S-Donner 3 points négatifs du plugin,S-Donner 3 améliorations à apporter au plugin
0,9,A,17/04/2025 08:47:23,Passerelle,Homme,"Duolingo, Kahoot!",4,4,4,17/04/2025 09:32:41,...,4,5,2,Oui,2,2,3,"Belle intégration, chouette concept, jolie UI",Le leaderboard ne s’est pas affiché pendant to...,"Bouton pour actualiser le leaderboard, amélior..."
1,23,A,17/04/2025 08:47:58,Passerelle,Homme,"Duolingo, Kahoot!",4,4,3,17/04/2025 09:32:42,...,3,3,5,Non,4,5,5,Donne un but (récompense) à la création de tes...,"Calcul des points à retravailler, j'ai reçu de...",Vérifier le calcul des points du leaderbord me...
2,36,B,17/04/2025 08:48:03,Passerelle,Homme,"Duolingo, Kahoot!",4,3,4,17/04/2025 10:07:39,...,4,4,4,Non,1,4,5,"Fun, stimulant, bien intégré",Buggé (achievements qui ne se valident pas inj...,Améliorer le système de connexion (une seule c...
3,8,B,17/04/2025 08:48:08,Passerelle,Homme,Duolingo,4,3,2,17/04/2025 10:07:55,...,5,4,1,Non,2,2,3,Clair\nFacile à prendre en main\nLudique,Trop d'achèvements\nJ'ai eu le sentiment de ne...,Moins d'achèvements (de 1 à 3 par catégories)\...
4,42,B,17/04/2025 08:48:15,BAC 3,Homme,"Duolingo, Kahoot!",2,2,1,17/04/2025 10:07:57,...,2,5,1,Non,5,5,1,Bien implémenté / amusant / sympa et cool,Score mal foutu / difficulté d’implémenter le ...,Le score car possibilité de dupliquer son scor...
5,47,A,17/04/2025 08:48:17,BAC 3,Homme,"Duolingo, Kahoot!, CodeCombat, GitHub Achievem...",4,2,2,17/04/2025 09:32:40,...,5,5,2,Non,2,5,5,"Pratique, pas compliqué a installé, motivant",Des gens trichants en réécrivant plusieurs foi...,"D’empêcher la triche, et rien de plus"
6,100,B,17/04/2025 08:48:18,BAC 3,Homme,Duolingo,2,1,3,17/04/2025 10:07:51,...,5,5,2,Oui,5,4,4,"Facile d'utilisation, Amusant, Motivant",On peut executer plusieurs fois le même test p...,"Pour chacun des types de tests évalués, Donner..."
7,92,B,17/04/2025 08:48:19,BAC 2,Femme,"Duolingo, Kahoot!",3,2,3,17/04/2025 10:07:11,...,2,5,5,Oui,2,5,5,"Visualisation de la progression, Temps de pris...",Je ne comprends pas toujours comment les point...,"Critères plus descriptifs, Liens éventuels ver..."
8,71,A,17/04/2025 08:48:21,Passerelle,Homme,"Duolingo, Kahoot!",4,3,2,17/04/2025 09:34:22,...,4,4,5,Non,3,5,4,"Assez simple d'utilisation,\nPlutôt intuitive\...",Tout ce qui est ajouts des points en run des t...,Voir points négatifs
9,119,A,17/04/2025 08:48:21,BAC 3,Femme,Duolingo,1,2,1,17/04/2025 09:33:37,...,2,4,5,Oui,1,5,5,C’est encourageant,Il n’y a pas de consignes definies,Mettre plus de consignes


# Fix user with non numeric identifiers

In [76]:
invalid_id_df = inner_merged_df[~inner_merged_df[id_key].str.isnumeric()].copy()
invalid_id_df.loc[:, 'Achievements Horodateur'] = pd.to_datetime(invalid_id_df['Achievements Horodateur'], format='%d/%m/%Y %H:%M:%S', utc=False)
invalid_id_df


,Identifiant,group,Demographic Horodateur,D-Année d'étude,D-Genre,D-Outils gamifiés déjà utilisés ?,D-Compétences java,D-Compétences en écriture de test,D-Motivation initiale à écrire des tests,Achievements Horodateur,...,"S-Clarté des règles du jeu (achievements, leaderboard)",S-Design et intégration dans IntelliJ,"S-Selon vous, quel mode est le plus fun ?",S-Le système de points et de progression vous a-t-il semblé juste ?,"S-J'ai été tenté d'écrire plus de tests pour augmenter mes points, même si les tests ne sont pas spécialement pertinents","S-Seriez-vous intéressé·e par l'utilisation d'outils gamifiés durant certains cours ?\n(Vérification et Validation, Génie Logiciel, ..)","S-Quel mode j'ai envie d'utiliser dans un cours où je peux être confronté à du test logiciel ? \n(Vérification et Validation, Génie Logiciel, ..)",S-Donner 3 points positifs du plugin,S-Donner 3 points négatifs du plugin,S-Donner 3 améliorations à apporter au plugin
17,iessarhi,Invalid,17/04/2025 08:48:37,BAC 3,Homme,"Duolingo, Kahoot!",3,2,1,2025-04-17 09:34:37,...,5,4,2,Non,4,5,5,"fun, drole à utiliser entre potes, donne de l'...","triche possible, certainement pas une bonne id...",Empecher la triche\n


# Strange data

## B group user first filled the achievements form

In [77]:
inner_merged_df['Achievements Horodateur'] = pd.to_datetime(inner_merged_df['Achievements Horodateur'], format='%d/%m/%Y %H:%M:%S', utc=False)
invalid_b_group = inner_merged_df[(inner_merged_df['group'] == 'B') & (pd.to_datetime(inner_merged_df['Achievements Horodateur']) < pd.to_datetime('2025-04-17 10:00:00'))][['Identifiant', 'group', 'Achievements Horodateur']]
invalid_b_group

,Identifiant,group,Achievements Horodateur
24,54,B,2025-04-17 09:33:22


## A group user first filled the leaderboard form

In [78]:
inner_merged_df['Leaderboard Horodateur'] = pd.to_datetime(inner_merged_df['Leaderboard Horodateur'], format='%d/%m/%Y %H:%M:%S', utc=False)
inner_merged_df[(inner_merged_df['group'] == 'A') & (pd.to_datetime(inner_merged_df['Leaderboard Horodateur']) < pd.to_datetime('2025-04-17 10:00:00'))][['Identifiant', 'group', 'Leaderboard Horodateur']]

,Identifiant,group,Leaderboard Horodateur


## Save to csv

In [79]:
inner_merged_df.to_csv(output_csv_file, index=False)
print(f"Merged forms data save at {output_csv_file}")

Merged forms data save at ../../generated/forms/data.csv
